In [2]:
import csv
import re
import os
import pandas as pd
import numpy as np  

# Đọc file data.csv

In [3]:
df = pd.read_csv("../data_csv/data.csv", encoding="utf-8")
df.head(2)

,id,name,address,lat,lon,poi_type,avg_stars,total_reviews,crowd,offerings,atmosphere,highlights,dining_options,children,accessibility,popular_for,opening_hours
0,0f9d2009-9436-46a4-b354-b0261898a39e,The Pub Coffee - Beer & Cocktail,"18A17 Tăng Nhơn Phú, Phước Long B, Quận 9, Thà...",10.829481,106.773785,"Cafe,Bar",4.9,181,Groups,"Alcohol, Beer, Cocktails, Coffee, Hard liquor","Casual, Cozy","Great beer selection, Great coffee, Live music...",Table service,NaN,NaN,NaN,"[{'day': 'Monday', 'hours': [{'start': '00:00'..."
1,02887955-963a-43ac-b0f7-355d7d7cfacf,Julieta,"C. Sta. Lucía, 9, Distrito Centro, 29008 Málag...",36.722011,-4.421780,Cafe,4.3,2053,"College students, Groups, Tourists","Alcohol, Beer, Coffee, Healthy options, Organi...","Casual, Cozy, Trendy","Great coffee, Great dessert, Great tea selection","Breakfast, Brunch, Lunch, Dessert, Seating, Ta...",Good for kids,"Wheelchair accessible entrance, Wheelchair acc...","Breakfast, Good for working on laptop","[{'day': 'Monday', 'hours': [{'start': '08:00'..."


# Trực quan dữ liệu

In [4]:
# Kiểm tra dữ liệu hiện tại của avg_stars và total_reviews
print("Kiểm tra dữ liệu avg_stars và total_reviews:")
print(f"Kiểu dữ liệu avg_stars: {df['avg_stars'].dtype}")
print(f"Kiểu dữ liệu total_reviews: {df['total_reviews'].dtype}")
print(f"Số lượng giá trị null trong avg_stars: {df['avg_stars'].isna().sum()}")
print(f"Số lượng giá trị null trong total_reviews: {df['total_reviews'].isna().sum()}")
print(f"\nMẫu dữ liệu:")
print(df[['name', 'avg_stars', 'total_reviews']].head(15))
print(f"\nCác giá trị unique của avg_stars: {df['avg_stars'].unique()[:20]}")
print(f"\nCác giá trị unique của total_reviews: {df['total_reviews'].unique()[:20]}")

Kiểm tra dữ liệu avg_stars và total_reviews:
Kiểu dữ liệu avg_stars: float64
Kiểu dữ liệu total_reviews: int64
Số lượng giá trị null trong avg_stars: 2
Số lượng giá trị null trong total_reviews: 0

Mẫu dữ liệu:
                                                 name  avg_stars  \
0                    The Pub Coffee - Beer & Cocktail        4.9   
1                                             Julieta        4.3   
2                  BAIETA Restaurant Saigon Thao Dien        4.5   
3                           Parroquia San Felipe Neri        4.7   
4                                              BÀ BAR        4.8   
5                                    Art Bar Viet Nam        4.9   
6   Ngỡ Cafe | Cà phê view đẹp Thủ Đức | Coffee ng...        4.7   
7                              TEEMAY COFFEE ROASTERS        5.0   
8                        La Mafia se sienta a la mesa        4.3   
9                                          Bar tocata        4.0   
10                           BiaCraft Xua

In [5]:
# Filter POI types with empty opening_hours
# Check for empty lists/strings or null values
empty_hours_mask = (df['opening_hours'] == '[]') | (df['opening_hours'] == '') | (df['opening_hours'].isna())
poi_types_with_empty_hours = df[empty_hours_mask]['poi_type'].unique()

print(f"POI Types with Empty Opening Hours ({len(poi_types_with_empty_hours)} types):")
print("="*50)
for poi_type in sorted(poi_types_with_empty_hours):
    count = len(df[(empty_hours_mask) & (df['poi_type'] == poi_type)])
    print(f"{poi_type}: {count} entries")

print("="*50)
print(f"\nTotal entries with empty opening_hours: {empty_hours_mask.sum()}")

POI Types with Empty Opening Hours (110 types):
Airport: 2 entries
Airport shuttle service,Minibus taxi service,Taxi service: 1 entries
Airport,Travel lounge: 1 entries
Amusement park: 1 entries
Andalusian restaurant,Spanish restaurant: 1 entries
Art center: 4 entries
Art gallery,Cultural center,Photography studio,Tourist attraction: 1 entries
Art museum: 1 entries
Art studio,Artistic handicrafts,Sculpture,Sculpture museum,Statuary: 1 entries
Auto repair shop,Auto parts market,Car wash: 1 entries
Bakery: 4 entries
Bakery,Brunch restaurant,Cafe,Homestay: 1 entries
Bakery,Hotel: 1 entries
Bar: 3 entries
Bar & grill: 1 entries
Beach: 13 entries
Beach clothing store: 1 entries
Beach clothing store,Costume store: 1 entries
Beach pavillion: 1 entries
Beach volleyball court: 1 entries
Boarding house,Cafe: 1 entries
Botanical garden: 1 entries
Building materials market: 1 entries
Bus station: 5 entries
Cafe: 6 entries
Capsule hotel,Hotel: 1 entries
Casino,Bar,Disco club,Event planner,Hotel,Tou

# Xử lí giá trị Null

**Xử lí cho avg_stars và total_reviews**

In [6]:
# Xử lý avg_stars và total_reviews theo yêu cầu
# Nếu avg_stars hoặc total_reviews không có giá trị hoặc 1 trong 2 có giá trị 0
# thì cả 2 trường sẽ được đưa về giá trị thấp nhất

# Chuyển đổi sang numeric, các giá trị không hợp lệ sẽ thành NaN
df['avg_stars'] = pd.to_numeric(df['avg_stars'], errors='coerce')
df['total_reviews'] = pd.to_numeric(df['total_reviews'], errors='coerce')

# Tìm giá trị thấp nhất của mỗi cột (không tính null và 0)
min_avg_stars = df[(df['avg_stars'].notna()) & (df['avg_stars'] > 0)]['avg_stars'].min()
min_total_reviews = df[(df['total_reviews'].notna()) & (df['total_reviews'] > 0)]['total_reviews'].min()

print(f"Giá trị thấp nhất của avg_stars (không tính null và 0): {min_avg_stars}")
print(f"Giá trị thấp nhất của total_reviews (không tính null và 0): {min_total_reviews}")

# Tạo mask để xác định các hàng cần cập nhật
# Điều kiện: avg_stars hoặc total_reviews là null, hoặc 1 trong 2 bằng 0
mask = (
    df['avg_stars'].isna() | 
    df['total_reviews'].isna() | 
    (df['avg_stars'] == 0) | 
    (df['total_reviews'] == 0)
)

print(f"\nSố lượng hàng cần cập nhật: {mask.sum()}")
print(f"Trước khi cập nhật:")
print(df[mask][['name', 'avg_stars', 'total_reviews']].head(15))

# Cập nhật cả 2 cột về giá trị thấp nhất
df.loc[mask, 'avg_stars'] = min_avg_stars
df.loc[mask, 'total_reviews'] = min_total_reviews

print(f"\nSau khi cập nhật:")
print(df[mask][['name', 'avg_stars', 'total_reviews']].head(15))

print(f"\nKiểm tra lại:")
print(f"Số lượng giá trị null trong avg_stars: {df['avg_stars'].isna().sum()}")
print(f"Số lượng giá trị null trong total_reviews: {df['total_reviews'].isna().sum()}")
print(f"Số lượng giá trị 0 trong avg_stars: {(df['avg_stars'] == 0).sum()}")
print(f"Số lượng giá trị 0 trong total_reviews: {(df['total_reviews'] == 0).sum()}")

Giá trị thấp nhất của avg_stars (không tính null và 0): 3.0
Giá trị thấp nhất của total_reviews (không tính null và 0): 1

Số lượng hàng cần cập nhật: 10
Trước khi cập nhật:
                                                   name  avg_stars  \
255                                Co.opmart Chu Van An        4.1   
257                        PICKLEBALL COLEGIO LA COLINA        NaN   
307                         GOPICK - 250 Lương Định Của        4.6   
597                            Tân Định Catholic Church        4.4   
681                      Nhim Sau Rieng Gelato & Coffee        4.8   
716                             Café Slow in the Garden        4.8   
746                                        Chợ Hạt Điều        4.2   
827                                        Ăn Vặt Bé Bơ        5.0   
877   BIA HƠI HÀ NỘI ĐẠI VIỆT | QUÁN BIA HƠI QUẬN 12...        3.8   
1452                                    parque infantil        NaN   

      total_reviews  
255               0  
257        

**Xử lí cho opening_hours**

In [7]:
import json
# Define default opening hours (24/7)
default_opening_hours = [
    {'day': 'Monday', 'hours': [{'start': '00:00', 'end': '23:59'}]},
    {'day': 'Tuesday', 'hours': [{'start': '00:00', 'end': '23:59'}]},
    {'day': 'Wednesday', 'hours': [{'start': '00:00', 'end': '23:59'}]},
    {'day': 'Thursday', 'hours': [{'start': '00:00', 'end': '23:59'}]},
    {'day': 'Friday', 'hours': [{'start': '00:00', 'end': '23:59'}]},
    {'day': 'Saturday', 'hours': [{'start': '00:00', 'end': '23:59'}]},
    {'day': 'Sunday', 'hours': [{'start': '00:00', 'end': '23:59'}]}
]

default_opening_hours_str = json.dumps(default_opening_hours)
print("Default opening hours:")
print(default_opening_hours_str)

Default opening hours:
[{"day": "Monday", "hours": [{"start": "00:00", "end": "23:59"}]}, {"day": "Tuesday", "hours": [{"start": "00:00", "end": "23:59"}]}, {"day": "Wednesday", "hours": [{"start": "00:00", "end": "23:59"}]}, {"day": "Thursday", "hours": [{"start": "00:00", "end": "23:59"}]}, {"day": "Friday", "hours": [{"start": "00:00", "end": "23:59"}]}, {"day": "Saturday", "hours": [{"start": "00:00", "end": "23:59"}]}, {"day": "Sunday", "hours": [{"start": "00:00", "end": "23:59"}]}]


In [8]:
# Fill empty opening_hours with default values
df.loc[empty_hours_mask, 'opening_hours'] = default_opening_hours_str

print(f"Updated {empty_hours_mask.sum()} entries with default opening hours")
print("\nSample of updated entries:")
print(df[df['poi_type'] == 'airport'][['poi_type', 'opening_hours']].head())

Updated 317 entries with default opening hours

Sample of updated entries:
Empty DataFrame
Columns: [poi_type, opening_hours]
Index: []


# Xuất ra csv

In [9]:
df.to_csv("../data_csv/data_cleaned.csv", index=False, encoding="utf-8")